In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.cluster import KMeans

In [2]:
%%time
train_df = pd.read_csv("../../01_input/train.csv")
test_df = pd.read_csv("../../01_input/test.csv")

CPU times: user 11.2 s, sys: 456 ms, total: 11.7 s
Wall time: 11.7 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [4]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

k-meansに備えてスケーリング

In [6]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [8]:
df_kmeans=pd.concat([x_train,x_test])
df_kmeans=scale.fit_transform(df_kmeans.as_matrix())


In [12]:
df_kmeans.shape

(400000, 200)

In [13]:
df_kmeans=pd.DataFrame(data=df_kmeans)

In [16]:
df_kmeans.describe()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
count,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,...,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05
mean,-2.643700e-16,-2.195744e-18,6.377576e-16,-7.879791e-16,1.107984e-15,1.934300e-16,-7.347678e-16,-3.405110e-17,6.525142e-17,1.559874e-15,...,-2.025302e-16,2.384148e-17,2.344805e-17,2.748561e-16,-8.259887e-16,1.667694e-17,7.366080e-17,-1.419941e-15,-5.297929e-16,-1.443415e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.449396e+00,-3.316829e+00,-3.258609e+00,-3.336174e+00,-3.705739e+00,-3.496534e+00,-3.691624e+00,-3.270060e+00,-3.236170e+00,-2.916839e+00,...,-3.798688e+00,-3.352922e+00,-3.881439e+00,-3.783430e+00,-2.964076e+00,-3.583790e+00,-3.028562e+00,-3.201785e+00,-3.179804e+00,-3.471882e+00
25%,-7.307891e-01,-7.650583e-01,-7.513572e-01,-7.569224e-01,-7.348989e-01,-7.809155e-01,-7.414728e-01,-7.597434e-01,-7.775203e-01,-7.676108e-01,...,-7.220310e-01,-7.560376e-01,-7.032163e-01,-6.862673e-01,-7.530434e-01,-7.191376e-01,-7.788340e-01,-7.104284e-01,-6.749404e-01,-7.568227e-01
50%,-4.949182e-02,6.558953e-03,-5.358401e-02,1.529691e-02,1.675131e-02,2.850624e-02,-2.721236e-02,-2.855547e-02,3.073859e-02,5.065246e-02,...,-6.673734e-03,-2.798683e-02,-2.020789e-02,2.103389e-02,-9.010985e-03,-2.079519e-02,2.012404e-02,-2.137595e-02,2.300735e-02,4.959887e-02
75%,6.842098e-01,7.356636e-01,6.802097e-01,7.488326e-01,7.281343e-01,7.618149e-01,6.841601e-01,7.486971e-01,7.960931e-01,8.239205e-01,...,6.938886e-01,6.855810e-01,6.935605e-01,7.172366e-01,7.643404e-01,6.798557e-01,7.783324e-01,7.431162e-01,7.260800e-01,7.847521e-01
max,3.835603e+00,2.967103e+00,3.276602e+00,3.122973e+00,3.453528e+00,2.836370e+00,3.506777e+00,3.435935e+00,2.961494e+00,2.904237e+00,...,3.763895e+00,3.064625e+00,4.377736e+00,3.744523e+00,3.171682e+00,3.276097e+00,2.940066e+00,3.653136e+00,3.544542e+00,3.051066e+00


In [18]:
df_kmeans_pred=df_kmeans
for i in range(2,15):
    print(i)
    clf=KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=300,
                           tol=0.0001,precompute_distances='auto', verbose=0,
                           random_state=None, copy_x=True, n_jobs=1)
    pred=clf.fit_predict(df_kmeans)
    df_kmeans_pred['kmeans_'+str(i)]=pred
df_kmeans_pred.head()

2
3
4
5
6
7
8
9
10
11
12
13
14


,0,1,2,3,4,5,6,7,8,9,...,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10,kmeans_11,kmeans_12,kmeans_13,kmeans_14
0,-0.573930,-1.275655,0.453774,-0.829761,0.236655,-0.537163,-0.338885,0.610620,-1.560352,-1.476589,...,0,4,2,0,1,9,0,0,6,4
1,0.273590,-0.623288,1.193406,-0.685231,0.793204,1.538372,0.241141,-0.001080,0.859932,0.418964,...,1,1,6,5,5,7,6,5,11,0
2,-0.677998,-0.276809,0.519141,0.537312,-0.305510,-0.511776,1.768200,-0.561777,-1.560142,-1.309985,...,4,2,1,6,6,1,1,11,9,6
3,0.128711,-0.129996,-0.666993,0.196935,0.930505,0.409577,0.497595,-0.471313,-1.842650,0.548599,...,4,2,3,7,3,5,4,1,3,11
4,-0.273969,0.035234,0.820234,-0.075620,0.740728,0.954270,0.610458,0.793242,1.795606,0.089243,...,4,2,3,7,3,5,4,1,3,2


In [27]:
df_kmeans_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10,kmeans_11,kmeans_12,kmeans_13,kmeans_14
0,-0.573930,-1.275655,0.453774,-0.829761,0.236655,-0.537163,-0.338885,0.610620,-1.560352,-1.476589,...,0,4,2,0,1,9,0,0,6,4
1,0.273590,-0.623288,1.193406,-0.685231,0.793204,1.538372,0.241141,-0.001080,0.859932,0.418964,...,1,1,6,5,5,7,6,5,11,0
2,-0.677998,-0.276809,0.519141,0.537312,-0.305510,-0.511776,1.768200,-0.561777,-1.560142,-1.309985,...,4,2,1,6,6,1,1,11,9,6
3,0.128711,-0.129996,-0.666993,0.196935,0.930505,0.409577,0.497595,-0.471313,-1.842650,0.548599,...,4,2,3,7,3,5,4,1,3,11
4,-0.273969,0.035234,0.820234,-0.075620,0.740728,0.954270,0.610458,0.793242,1.795606,0.089243,...,4,2,3,7,3,5,4,1,3,2


In [48]:
train_num=len(x_train)
test_num=len(x_test)

df_train_kmeans=df_kmeans_pred.head(train_num)
df_test_kmeans=df_kmeans_pred.tail(test_num)

In [57]:
for i in df_train_kmeans.columns:
    if 'kmeans' in str(i):
        print(i)
    else:
        df_train_kmeans=df_train_kmeans.drop(i, axis=1)
        df_test_kmeans=df_test_kmeans.drop(i, axis=1)

kmeans_2
kmeans_3
kmeans_4
kmeans_5
kmeans_6
kmeans_7
kmeans_8
kmeans_9
kmeans_10
kmeans_11
kmeans_12
kmeans_13
kmeans_14


In [58]:
df_train_kmeans.head()

,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10,kmeans_11,kmeans_12,kmeans_13,kmeans_14
0,1,0,2,0,4,2,0,1,9,0,0,6,4
1,0,1,1,1,1,6,5,5,7,6,5,11,0
2,0,1,3,4,2,1,6,6,1,1,11,9,6
3,0,1,3,4,2,3,7,3,5,4,1,3,11
4,0,1,3,4,2,3,7,3,5,4,1,3,2


In [59]:
df_test_kmeans.head()

,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10,kmeans_11,kmeans_12,kmeans_13,kmeans_14
200000,0,1,1,1,1,6,5,5,7,6,5,11,0
200001,0,1,3,4,2,1,6,6,1,1,11,9,6
200002,0,1,1,1,1,6,5,5,7,6,5,11,0
200003,0,1,1,1,1,6,5,7,2,2,4,1,13
200004,0,1,3,4,2,1,6,6,1,1,11,9,6


In [60]:
df_train_kmeans.to_csv('../../02_feature/102_train_kmeans.csv',index=False)
df_test_kmeans.to_csv('../../02_feature/102_test_kmeans.csv',index=False)